# Estimation de la masse du trou noir au centre de la galaxie NVSS J201943-364542: des ajustements à la masse

In [1]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [2]:
import astropy.constants as cst
import astropy.units as u
from astropy.io import fits
from astropy.cosmology import Planck15 as cosmo
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
plt.ion()

### Valeurs de départ

Les ajustements précédents nous donnent pour O ɪɪɪ et Hα : $\lambda_\mathrm{obs}$ et $\sigma$ (en Å), ainsi que $a$ et $b$ (en erg s⁻¹ cm⁻² arcsec⁻² Å⁻¹), avec à chaque fois les incertitudes associées (notées `d_valeur` par convention).

In [3]:
# centre de Hα (Å)
cen_Ha = 20462.8
d_cen_Ha = 2
# largeur de Hα (Å)
wid_Ha = 200
d_wid_Ha = 2
# amplitude de Hα (erg s⁻¹ cm⁻² arcsec⁻² Å⁻¹)
a_Ha =1e-19
d_a_Ha = 0.1e-19
# fond (constant) pour Hα (erg s⁻¹ cm⁻² arcsec⁻² Å⁻¹)
b_Ha = 6e-20
d_b_Ha = 0.1e-20
# centre de Oɪɪɪ (Å)
cen_o3 = 15618.5
d_cen_o3 = 2

Par ailleurs, on a les longueurs d’onde au repos suivantes :

In [4]:
# centre au repos de Hα (Å)
cen0_Ha = 6563.
# centre au repos de Oɪɪɪ (Å)
cen0_o3 = 5007.

### Démarche

Pour calculer la masse du trou noir, on utilise l’équation établie par Green & Ho (2005) :


$$
    M_\text{BH} = \left({2.0}^{+{0.4}}_{-{0.3}}\right)
      \times {10^6} 
      \left(\frac{L_{\textrm{H}\alpha}}{{10^{42}}\text{erg s⁻¹}}\right)^{{0.55\pm0.02}}
      \left(\frac{\text{FWHM}_{\textrm{H}\alpha}}{{10^{3}}\text{km s⁻¹}}\right)^{{2.06\pm0.06}}
      M_\odot.
$$

Il nous faut donc calculer :

- $L_{\textrm{H}\alpha}$, la luminosité absolue en erg s⁻¹ dans la raie Hα,
- $\text{FWHM}_{\textrm{H}\alpha}$, la largeur à mi-hauteur (full width at half maximum, FWHM) en km s⁻¹ de la raie Hα.

Pour calculer ces grandeurs, nous aurons besoin du redshift $z$, du flux par unité de surface angulaire $F_s$, de la surface angulaire $S_\text{galaxie}$, du flux $F$, et de la distance de luminosité $d_L$ de la galaxie.

### Redshift calculé pour O ɪɪɪ et Hα

$z = \frac{\lambda_\mathrm{obs} - \lambda_0}{\lambda_0}$

$\delta z = \frac{\delta\lambda_\mathrm{obs}}{\lambda_0}$

In [5]:
z_o3 = (cen_o3 - cen0_o3) / cen0_o3
d_z_o3 = d_cen_o3 / cen0_o3

z_Ha = (cen_Ha - cen0_Ha) / cen0_Ha
d_z_Ha = d_cen_Ha / cen0_Ha

On estime $z$ à partir de la raie de Oɪɪɪ, qui est plus fine, afin d’avoir une valeur plus précise. 

Par contre, on estimera les autres valeurs pour la raie de Hα.

In [6]:
z = z_o3
d_z = d_z_o3

### Flux de surface

C’est le flux par unité de surface angulaire (observée sur le ciel) émis par la galaxie. C’est simplement l’intensité intégrée sous la raie de Hα :

$F_s = a \sigma \sqrt{2\pi}$

$\delta F_s = \sqrt{(\delta a / a)^2 + (\delta \sigma / \sigma)^2} F_s$

On l’exprime en erg s⁻¹ cm⁻² arcsec⁻².

In [7]:
F_s = a_Ha * wid_Ha * np.sqrt(2 * np.pi) # erg s⁻¹ cm⁻² arcsec⁻²
d_F_s = np.sqrt((d_a_Ha / a_Ha)**2 + (d_wid_Ha / wid_Ha)**2) * F_s

### Flux

Nous souhaitons maintenant déterminer le flux total provenant de la galaxie. Pour ce faire, il faut intégrer **angulairement** sur l’ensemble des pixels de notre image couverts par la galaxie. Pour ce faire, on compte le nombre de pixels occupés par la galaxie sur l’image en Hα faite en 2e séance, et on convertit ce nombre de pixels en surface angulaire sur le ciel. 

Pour aller plus vite, on supposera que la surface de la galaxie vaut $S_\text{galaxie} = 0.88~\text{arcsec²}$. (C’est la valeur mesurée par Collet et al., 2016.)

On peut alors calculer le flux total reçu de la galaxie :

$F = F_s \times S_\text{galaxie}$

On prend $\delta F = \delta F_s \times S_\text{galaxie}$.

On l’exprime en erg s⁻¹ cm⁻².

In [8]:
S_galaxie = 0.88 # arcsec²

F = F_s * S_galaxie # erg s⁻¹ cm⁻² arcsec⁻²
d_F = d_F_s * S_galaxie

### Distance de luminosité

Pour calculer la luminosité absolue de la galaxie, il « suffit » d’appliquer la formule $L = 4\pi d^2 F$. Mais quelle valeur de distance prendre, à des échelles cosmologiques ?

On utilise la distance de luminosité, $d_L$, qui se calcule avec la formule suivante :

$d_L = (1+z) \frac{c}{H_0} \int_0^z \frac{\textrm{d}z^\prime}{\sqrt{\Omega_m (1+z^\prime)^3 + \Omega_\Lambda}}$

$\delta d_L = \left( \frac{d_L}{1+z} + (1+z) \frac{c}{H_0} \frac{1}{\sqrt{\Omega_m (1+z)^3 + \Omega_\Lambda}} \right) \delta z$

avec $H_0 \approx 70~\text{km Mpc⁻¹ s⁻¹}$, $\Omega_m \approx 0.3$, et $\Omega_\Lambda \approx 0.7$.

Puisque cette intégrale n’a pas de solution analytique, on utilise la fonction `luminosity_distance` du module `astropy.cosmology.Planck15`.

De plus, il faut penser à convertir ce résultat en centimètres.

In [9]:
d_l = cosmo.luminosity_distance(z) # Mpc (type astropy.units.quantity contenant valeur ET unité)
d_d_l = (d_l / (1+z) + (1 + z) * (cst.c / cosmo.H0) * 1 / (np.sqrt(cosmo.Odm0 * (1 + z)**3 + cosmo.Ode0))) * d_z

d_l = d_l.to('cm').value # cm
d_d_l = d_d_l.to('cm').value # cm

### Luminosité absolue

En connaissant $d_L$, on peut calculer la luminosité absolue de la galaxie :

$L = 4\pi d_L^2 F$

$\delta L = \sqrt{(2 \delta d_L / d_L)^2 + (\delta F / F)^2} L$

Cette luminosité s’exprime en erg s⁻¹.

In [10]:
L = 4 * np.pi * d_l**2 * F # erg s⁻¹
d_L = L * np.sqrt((2 * d_d_l / d_l)**2 + (d_F / F)**2)

### Largeur à mi-hauteur

Enfin, la dernière grandeur dont on a besoin pour estimer la masse du trou noir est la largeur à mi-hauteur, exprimée en km s⁻¹.

Largeur à mi-hauteur en Å :

$\Delta \lambda = 2\sqrt{2~\mathrm{ln}2}\sigma \approx 2.35 \sigma$

Largeur à mi-hateur en km s⁻¹ :

$\text{FWHM}_{\mathrm{H}\alpha} = \Delta v = \frac{\Delta\lambda}{\lambda_\text{obs}} \times c$

In [11]:
c = cst.c.to('km/s').value # c en km s⁻¹

delta_lda = 2 * np.sqrt(2 * np.log(2)) * wid_Ha # Å
d_delta_lda = d_wid_Ha / wid_Ha * delta_lda

delta_v = delta_lda / cen0_Ha * c # km s⁻¹
d_delta_v = np.sqrt((d_delta_lda / delta_lda)**2 + (d_cen_Ha / cen0_Ha)**2) * delta_v

### Masse du trou noir (enfin !)



$$
    M_\text{BH} = \left({2.0}^{+{0.4}}_{-{0.3}}\right)
      \times {10^6} 
      \left(\frac{L_{\textrm{H}\alpha}}{{10^{42}}\text{erg s⁻¹}}\right)^{{0.55\pm0.02}}
      \left(\frac{\text{FWHM}_{\textrm{H}\alpha}}{{10^{3}}\text{km s⁻¹}}\right)^{{2.06\pm0.06}}
      M_\odot.
$$

Pour trouver l’expression de $δM_\text{BH}$, on fait la dérivée logarithmique de $M_\text{BH}$.

In [12]:
m0 = 2e6 # M_s
d_m0 = 0.35e6 # M_s (en réalité, on devrait prendre -0.3e6, +0.4e6)
alpha = 0.55
d_alpha = 0.02
beta = 2.06
d_beta = 0.06
L0 = 1e42 # erg s⁻¹
delta_v0 = 1e3 # km s⁻¹

M = m0 * \
    (L / L0)**alpha * \
    (delta_v / delta_v0)**beta # M☉
d_M = M * np.sqrt(
    (d_m0 / m0)**2 + \
    (d_alpha * np.log(L / L0))**2 + \
    (alpha * d_L / L)**2 + \
    (d_beta * np.log(delta_v / delta_v0))**2 + \
    (beta * d_delta_v / delta_v)**2
    )

### Résultats

In [13]:
def power_of_ten(exp):
    exp_str = '{:+d}'.format(exp)
    exp_converter = dict(zip('1234567890+−-', '¹²³⁴⁵⁶⁷⁸⁹⁰⁺⁻⁻'))
    exp_str = ''.join([exp_converter[s] for s in exp_str])
    exp_str = '10' + exp_str
    return exp_str

def format_quantity(name, val, unc, unit, unc_digits=1):
    ''' Format a physical quantity with its uncertainty and unit.
    
    Parameters
    ==========
    name : str
    val : float
        value
    unc : float
        uncertainty
    unit : str
    unc_digits : int (default: 1)
        Number of significant digits to use for the uncertainty.
        
    Returns
    =======
    A string representing the input quantity
    
    Example
    =======
    >>> format_quantity('x', 1.1238e3, 2.18e2, 'cm')
    'x = (1.1 ± 0.2)e3 cm'
    >>> format_quantity('x', 1.1238e3, 2.18e2, 'cm', unc_digits=2)
    'x = (1.12 ± 0.22)e3 cm'
    '''
    exponent_val = int(np.floor(np.log10(val)))
    exponent_unc = int(np.floor(np.log10(unc)))
    norm_val = val / 10**exponent_val
    norm_unc = unc / 10**exponent_val
    digits = exponent_val - exponent_unc + unc_digits - 1
    num = '{norm_val:.{digits}f} ± {norm_unc:.{digits}f}'
    if exponent_val != 0:
        num = '({}) {}'.format(num, power_of_ten(exponent_val))
    num = num.format(**locals())
    return '{name} = {num} {unit}'.format(**locals())

In [14]:
values = (
    ('z', z, d_z, ''),
    ('d_L', d_l, d_d_l, 'cm'),
    ('F_s', F_s, d_F_s, 'erg s⁻¹ cm⁻² arcsec⁻²'),
    ('F', F, d_F, 'erg s⁻¹ cm⁻²'),
    ('L', L, d_L, 'erg s⁻¹'),
    ('Δλ', delta_lda, d_delta_lda, 'Å'),
    ('Δv', delta_v, d_delta_v, 'km s⁻¹'),
    ('M', M, d_M, 'M☉')
    )

for v in values:
    print(format_quantity(*v))

z = 2.1193 ± 0.0004 
d_L = (5.277 ± 0.001) 10⁺²⁸ cm
F_s = (5.0 ± 0.5) 10⁻¹⁷ erg s⁻¹ cm⁻² arcsec⁻²
F = (4.4 ± 0.4) 10⁻¹⁷ erg s⁻¹ cm⁻²
L = (1.5 ± 0.2) 10⁺⁴² erg s⁻¹
Δλ = (4.71 ± 0.05) 10⁺² Å
Δv = (2.15 ± 0.02) 10⁺⁴ km s⁻¹
M = (1.4 ± 0.4) 10⁺⁹ M☉
